In [33]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
      .master("local[1]")
      .appName("SparkByExamples.com")
      .getOrCreate();

val df_transactions = spark.read.option("header",true).csv("transactions.csv")
val df_contracts = spark.read.option("header", true).csv("contracts.csv")

val df_transactions_read = df_transactions.na.replace("discount_percentage", Map("null" -> "0"))

df_transactions_read.show()
df_contracts.show()

+---------------+---------+------------+-------------------+
|transaction_id |client_id|total_amount|discount_percentage|
+---------------+---------+------------+-------------------+
|              1|     3545|        3000|               6.99|
|              2|     3545|        4500|               0.45|
|              3|     3509|       69998|                  0|
|              4|     3510|           1|                  0|
|              5|     4510|          34|                 40|
+---------------+---------+------------+-------------------+

+-----------+---------+----------------+----------+---------+
|contract_id|client_id|     client_name|percentage|is_active|
+-----------+---------+----------------+----------+---------+
|          3|     3545|  Magazine Luana|      2.00|     true|
|          4|     3545| Magazine Luana |      1.95|    false|
|          5|     3509|Lojas Italianas |         1|     true|
|          6|     3510|      Carrerfive|      3.00|     true|
+-----------+---

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@400b5225
df_transactions: org.apache.spark.sql.DataFrame = [transaction_id : string, client_id: string ... 2 more fields]
df_contracts: org.apache.spark.sql.DataFrame = [contract_id: string, client_id: string ... 3 more fields]
df_transactions_read: org.apache.spark.sql.DataFrame = [transaction_id : string, client_id: string ... 2 more fields]


In [34]:
val df_liquid_value = df_transactions_read.selectExpr("client_id", "total_amount - (total_amount * (discount_percentage / 100)) as liquid_value")
df_liquid_value.show()

+---------+------------+
|client_id|liquid_value|
+---------+------------+
|     3545|      2790.3|
|     3545|     4479.75|
|     3509|     69998.0|
|     3510|         1.0|
|     4510|        20.4|
+---------+------------+



df_liquid_value: org.apache.spark.sql.DataFrame = [client_id: string, liquid_value: double]


In [49]:
df_liquid_value.join(df_contracts, df_liquid_value("client_id") === df_contracts("client_id"), "inner").filter(df_contracts("is_active") === "true").selectExpr("sum(liquid_value * (percentage / 100)) as total").show()

+-----------------+
|            total|
+-----------------+
|845.4110000000001|
+-----------------+

